# oneM2M - Subscriptions and Notifications - Notification Server

This notebook runs a small webserver to receive notifications from a CSE.

Please note that it is necessary to run this server in a separate notebook. Please refer to the second notebook on this topic for the requests.

**Note**: The server can only be run once! If you need to restart the server, you also need to **interrupt and restart** the Jupyter kernel.


## Setup and Run a Notification Server

In order to subscribe to notifications one must run a notification server that receives two kind of requests:

- Whenever a new &lt;Subscription> is created a verification request is send to the notification server to check the validity of the subscription.
- For every monitored change of a resource the server is contacted and the changes are pushed to the notification server.m

For a simple implementation (always acknowledge the requests described above), the notification server must set the response header field *X-M2M-RSC* to *2000*.



In [ ]:
from init import *
from http.server import HTTPServer, BaseHTTPRequestHandler

class SimpleHTTPRequestHandler(BaseHTTPRequestHandler):
        
    def do_POST(self):
        # Construct return header
        self.send_response(200)
        self.send_header('X-M2M-RSC', '2000')
        self.end_headers()

        # Get headers and content data
        length = int(self.headers['Content-Length'])
        contentType = self.headers['Content-Type']
        post_data = self.rfile.read(length)
        
        # Print the content data
        printmd('### Notification')
        #print (self.headers)
        printJSON(post_data.decode('utf-8'))


httpd = HTTPServer(('', notificationPort), SimpleHTTPRequestHandler)
printmd('**starting server & waiting for connections**')
httpd.serve_forever()

## The following attributes can be part of notification:

- **vrq** : When this attribute is present and set to *true* then a request is a Verification Requst.
- **sud** : A Subscription Deletion attribute indicates whether this request signals the deletion of a subscription.
- **nev** : This attribute indicates and represents that a request is a Notification Event.
- **rep** : The Representation attribute contains the resource(s) that was changed and caused the notification. Usually it contains the whole resource for further processing by the notification server.
- **sur** : The Subscription Resource attribute contains the resource identifier of the assocsubiated &lt;Subscription>.